In [2]:
import duckdb

In [3]:
# Connect to DuckDB (creates an in-memory database by default)
con = duckdb.connect()

In [4]:
# Drop the Sales table if it exists
con.execute('DROP TABLE IF EXISTS Sales')

# Create the Sales table
con.execute('''
CREATE TABLE Sales (
    sales_id INT,
    customer_id INT,
    product_id INT,
    sale_date DATE,
    quantity INT,
    total_amount DECIMAL
)
''')

# Drop the Customers table if it exists
con.execute('DROP TABLE IF EXISTS Customers')

# Create the Customers table
con.execute('''
CREATE TABLE Customers (
    customer_id INT,
    customer_name VARCHAR,
    sales_region VARCHAR,
    sign_up_date DATE
)
''')

# Drop the Products table if it exists
con.execute('DROP TABLE IF EXISTS Products')

# Create the Products table
con.execute('''
CREATE TABLE Products (
    product_id INT,
    product_name VARCHAR,
    category VARCHAR,
    price DECIMAL
)
''')


In [5]:
# Optional: Verify the tables are created
con.execute('SHOW TABLES').fetchall()

[('Customers',), ('Products',), ('Sales',)]

In [6]:
# Insert data into the Customers table
con.execute('''
INSERT INTO Customers VALUES
    (1, 'Alice', 'North', '2024-01-15'),
    (2, 'Bob', 'South', '2024-02-10'),
    (3, 'Charlie', 'East', '2024-03-05'),
    (4, 'David', 'West', '2024-04-01'),
    (5, 'Eve', 'Central', '2024-05-15'),
    (6, 'Frank', 'North', '2024-06-20'),
    (7, 'George', 'South', '2024-09-25'),
    (8, 'Helen', 'East', '2024-10-30')
''')

# Insert data into the Products table
con.execute('''
INSERT INTO Products VALUES
    (101, 'Laptop', 'Electronics', 1200.00),
    (102, 'Smartphone', 'Electronics', 800.00),
    (103, 'Tablet', 'Electronics', 400.00),
    (104, 'Desktop', 'Electronics', 1500.00),
    (105, 'Smartwatch', 'Electronics', 300.00),
    (106, 'Headphones', 'Electronics', 100.00),
    (107, 'Speaker', 'Electronics', 200.00),
    (108, 'Mouse', 'Electronics', 50.00)
''')

# Insert data into the Sales table
con.execute('''
INSERT INTO Sales VALUES
    (1, 1, 101, '2023-09-10', 2, 2400.00),
    (2, 2, 102, '2023-09-11', 1, 800.00),
    (3, 3, 103, '2023-09-12', 3, 1200.00),
    (4, 4, 104, '2023-09-13', 1, 1500.00),
    (5, 5, 105, '2023-09-14', 2, 600.00),
    (6, 6, 106, '2023-09-15', 3, 300.00),
    (7, 7, 107, '2023-09-16', 2, 400.00),
    (8, 8, 108, '2023-09-17', 4, 200.00)
''')

In [7]:
# Optional: Verify the data is inserted
customers_data = con.execute('SELECT * FROM Customers').fetchdf()
products_data = con.execute('SELECT * FROM Products').fetchdf()
sales_data = con.execute('SELECT * FROM Sales').fetchdf()

In [8]:
customers_data.to_csv('../data/q2/customers.csv', index=False)
products_data.to_csv('../data/q2/products.csv', index=False)
sales_data.to_csv('../data/q2/sales.csv', index=False)

# Solutions

In [13]:
%load_ext magic_duckdb

In [16]:
%%dql -t df
INSTALL httpfs;

,Success


In [17]:
%%dql -t df
LOAD  httpfs;

,Success


In [22]:
%%dql
CREATE OR REPLACE TABLE customers AS 
SELECT * FROM read_csv_auto('https://raw.githubusercontent.com/MarkPhamm/Clipboard-Case-Study/main/data/q2/customers.csv');

,Count
0,8


In [24]:
%%dql
CREATE OR REPLACE TABLE products AS 
SELECT * FROM read_csv_auto('https://raw.githubusercontent.com/MarkPhamm/Clipboard-Case-Study/main/data/q2/products.csv');

,Count
0,8


In [25]:
%%dql
CREATE OR REPLACE TABLE sales AS 
SELECT * FROM read_csv_auto('https://raw.githubusercontent.com/MarkPhamm/Clipboard-Case-Study/main/data/q2/sales.csv');

,Count
0,8


In [26]:
%%dql
SELECT * FROM customers
LIMIT 5

,customer_id,customer_name,sales_region,sign_up_date
0,1,Alice,North,2023-01-15
1,2,Bob,South,2023-02-10
2,3,Charlie,East,2023-03-05
3,4,David,West,2023-04-01
4,5,Eve,Central,2023-05-15


In [29]:
%%dql
SELECT * FROM sales
LIMIT 5

,sales_id,customer_id,product_id,sale_date,quantity,total_amount
0,1,1,101,2023-09-10,2,2400.0
1,2,2,102,2023-09-11,1,800.0
2,3,3,103,2023-09-12,3,1200.0
3,4,4,104,2023-09-13,1,1500.0
4,5,5,105,2023-09-14,2,600.0


In [28]:
%%dql
SELECT * FROM products
LIMIT 5

,product_id,product_name,category,price
0,101,Laptop,Electronics,1200.0
1,102,Smartphone,Electronics,800.0
2,103,Tablet,Electronics,400.0
3,104,Desktop,Electronics,1500.0
4,105,Smartwatch,Electronics,300.0


In [30]:

%%dql
SELECT 
    c.customer_name,
    p.product_name,
    s.total_amount
FROM 
    Sales s
JOIN 
    Customers c ON s.customer_id = c.customer_id
JOIN 
    Products p ON s.product_id = p.product_id
WHERE 
    s.sale_date >= CURRENT_DATE - INTERVAL '30 days'
ORDER BY 
    s.sale_date DESC;

,customer_name,product_name,total_amount
